In [1]:
from google.colab import drive
drive.mount('/content/drive')
% cd drive/MyDrive/thesis
! git clone https://www.github.com/pafitis/thesis

Mounted at /content/drive
[Errno 2] No such file or directory: 'drive/MyDrive/thesis'
/content
Cloning into 'thesis'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 242 (delta 131), reused 173 (delta 63), pack-reused 0
Receiving objects: 100% (242/242), 17.54 MiB | 23.82 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [2]:
! pip install spacy-nightly

     |████████████████████████████████| 12.7 MB 185 kB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 456 kB 67.7 MB/s 
     |████████████████████████████████| 621 kB 73.3 MB/s 
     |████████████████████████████████| 9.1 MB 13.8 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires catalogue<1.1.0,>=0.0.7, but you have catalogue 2.0.4 wh

In [3]:
!python -m spacy download en_core_web_trf

2021-08-04 22:33:20.877353: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 459.7 MB 1.2 MB/s 
     |████████████████████████████████| 6.3 MB 7.0 MB/s 
     |████████████████████████████████| 2.2 MB 67.5 MB/s 
     |████████████████████████████████| 998 kB 67.5 MB/s 
     |████████████████████████████████| 3.3 MB 73.5 MB/s 
     |████████████████████████████████| 895 kB 3.4 MB/s 
  Created wheel for en-core-web-trf: filename=en_core_web_trf-3.0.0-py3-none-any.whl size=459708991 sha256=2c5bd743ed57abe06a94b7b0c3e462a89c56ef3f819415c0823f15fad8fc7991
  Stored in directory: /tmp/pip-ephem-wheel-cache-pr3c6les/wheels/be/d9/92/c65214265163256ecc7588e6468c43feb86424a69bfe2fddb9
Successfully built en-core-web-trf
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
✔ Download and installatio

In [6]:
import sys
sys.path.append('/content/drive/MyDrive/thesis')
sys.path.append('/content/thesis')

In [5]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/thesis',
 '/content/thesis']

In [5]:
import pandas as pd
import numpy as np
import pickle, logging, spacy, sys, os, json, requests
import matplotlib.pyplot as plt

from helpers.classes import Collection
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

from helpers.cloze_generation import \
    generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

# from helpers.language_modelling import \
    # find_nth_substring, multitoken_prediction, check_model, run_language_model
from helpers.t5_language_model import \
    run_t5, check_t5

In [4]:
% cd thesis

/content/thesis


In [8]:
! pip install pickle5

     |████████████████████████████████| 132 kB 7.0 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219327 sha256=dcfc03ad48a71ca815a18d824b1270434ad54d9d095b62f52f9590f9e67facff
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [8]:
import pickle5 as pickle

In [9]:
# DATASETS = os.listdir('datasets')

# with open('pickles/collection_20210624_194932.pkl', 'rb') as f:
#     collection = pickle.load(f)

# # I should completely remove the other bulletins
# # for now this has to do...
# bulletin_names = list(collection.bulletins.keys())
# # target bulletins are those with usable related datasets and main points
# target_bulletins = []
# dictionary = dict()
# for bulletin in bulletin_names:
#     if len(collection.bulletins.get(bulletin).get('main-points')) and len(collection.bulletins.get(bulletin).get('related-datasets')) > 0:
#         target_bulletins.append(bulletin)



with open('pickles/dataset_20210625_184837.pkl', 'rb') as f:
    df = pickle.load(f)

In [10]:
df.head()

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...
3,businessindustryandtrade/business/businessserv...,date_and_percent,"Out of the 12 UK regions, 8 regions experience...",[/businessindustryandtrade/business/businessse...
4,businessindustryandtrade/business/businessserv...,date_and_percent,"West Midlands, Yorkshire and The Humber, Scotl...",[/businessindustryandtrade/business/businessse...


# T5

In [12]:
! pip install SentencePiece

     |████████████████████████████████| 1.2 MB 9.3 MB/s 


In [11]:
from transformers import T5Config, T5ForConditionalGeneration, T5Tokenizer
import torch

model_name = 't5-base'

device = 'cuda:0' if torch.cuda.is_available else 'cpu'
config = T5Config.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, config=config).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name, config=config)

In [17]:
test_df = df[:3]

In [18]:
test_df

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...


In [12]:
results, entity_set, entities = run_t5(df, model, tokenizer, True)

  0%|          | 0/2015 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
100%|██████████| 2015/2015 [1:43:55<00:00,  3.09s/it]


TypeError: ignored

In [16]:
entity_set

NameError: ignored

In [14]:
df

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...
3,businessindustryandtrade/business/businessserv...,date_and_percent,"Out of the 12 UK regions, 8 regions experience...",[/businessindustryandtrade/business/businessse...
4,businessindustryandtrade/business/businessserv...,date_and_percent,"West Midlands, Yorkshire and The Humber, Scotl...",[/businessindustryandtrade/business/businessse...
...,...,...,...,...
2010,peoplepopulationandcommunity/wellbeing/article...,date_and_percent,Trust in others in their neighbourhood was hig...,[/peoplepopulationandcommunity/wellbeing/datas...
2011,peoplepopulationandcommunity/wellbeing/article...,date_and_percent,Trust in others in their neighbourhood was hig...,[/peoplepopulationandcommunity/wellbeing/datas...
2012,peoplepopulationandcommunity/wellbeing/article...,date_and_percent,People in higher managerial occupations were m...,[/peoplepopulationandcommunity/wellbeing/datas...
2013,peoplepopulationandcommunity/wellbeing/article...,date_and_percent,Around 6 in 10 people (61%) reported feeling s...,[/peoplepopulationandcommunity/wellbeing/datas...
